# Imports

In [ ]:
import numpy as np
import cv2
import os
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import time
import pandas
from sklearn.metrics.pairwise import euclidean_distances

# Functions

In [ ]:
def image_retrieval_k(train_data, test_data, train_names, test_names, train_images_as_array, test_images_as_array, k=10, view_option=1):
    avg_precisions = []
    avg_recalls = []
    precisionsatk = []
    count = 0
    
    for idx, query in enumerate(test_data):
        
        all_precisions = []
        all_recalls = []
        precisions = []
        recalls = []

        # Finding the euclidean distance from the query image and sorting them into index
        query = query.reshape((1, -1))
        D = euclidean_distances(train_data, query).squeeze()
        index = np.argsort(D)
        
        # Finding the index of the last correct image in the sorted index to iter to
        last_correct_image_idx = 0
        for i in range(len(index)):
            if train_names[index[i]] == test_names[idx]:
                last_correct_image_idx = i
        
        # make sure we iter to k (for precision@k) if all correct images are found before k
        if k > last_correct_image_idx:
            last_correct_image_idx = k+1
        
        # Itering through all images untill we get to k or last correct image to compute AP
        for kk in range(1, last_correct_image_idx+2):
            TP = 0
            FP = 0
            FN = 0
            
            # Finding the correct amount of images in the training set
            correct_count = 0
            for ind in index:
                if train_names[ind] == test_names[idx]:
                    correct_count += 1
            sized_index = index[:kk]
            
            # Find TP FP FN
            for ind in sized_index:
                if train_names[ind] == test_names[idx]:
                    TP += 1
                else:
                    FP += 1
            FN = correct_count - TP
            
            # If the last k image is a correct image we add precision to the list
            if train_names[sized_index[-1]] == test_names[idx]:
                precisions.append(TP/(TP+FP))
                recalls.append(TP/(TP+FN))

            # Adding all precisions and recalls to a seperate list
            all_precisions.append(TP/(TP+FP))
            all_recalls.append(TP/(TP+FN))
        
     
        # Solving AP, AR and precision@k
        avg_precisions.append(np.average(precisions))
        avg_recalls.append(np.average(all_recalls))
        precisionsatk.append(all_precisions[k-1])
        
        # View options
        if view_option == 0:
            count += 1 
            clear_output(wait=True)
            print("Percentage Complete: {}".format((count/len(test_data))*100))
        elif view_option == 1:
            # creating an array of the top k similar images
            top_k_images = [test_images_as_array[idx]]
            for i in range(0,k):
                top_k_images.append(train_images_as_array[index[i]])

            fig, axes = plt.subplots(1, k+1, figsize=(200/k, 200/k))
            for i, (image, ax) in enumerate(zip(top_k_images, axes.ravel())):
                ax.imshow(image)
                if i == 0:
                    ax.set_title("Query: {}".format(test_names[idx]))
                else:
                    ax.set_title(train_names[sized_index[i-1]])
                ax.axis("off")
            plt.show()
            
            print("Label: {}".format(test_names[idx]))
            print("Average Precision for query {}: ".format(idx), avg_precisions[-1])
            print("Precision@k for query {}: ".format(idx), precisionsatk[-1])
            print("\n")
        
    return avg_precisions, avg_recalls, precisionsatk

In [ ]:
def save_data_to_csv(_precisionsatk, _AP, _k, _dataset_name):
    data = {'Precision@k': _precisionsatk, 'Average Precision': _AP}
    df = pandas.DataFrame(data=data)
    pandas.set_option("display.max_rows", 500, "display.max_columns", 4)
    df.to_csv('{}-metrics_k={}.csv'.format(_dataset_name, _k))

In [ ]:
def load_images_in(images_path, gt_path, end_fname):
    
    image_paths = []
    names = []
    image_dict = {}
    colour = []
    seen_names = []
    
    for filename in sorted(os.listdir(gt_path)):
        if filename.endswith(end_fname[0]) or filename.endswith(end_fname[1]):

            # Saving filename
            tmp = filename.split(".")[0].split("_")
            if len(tmp) == 4:
                name = tmp[0]+"_"+tmp[1]
            elif len(tmp) == 3:
                name = tmp[0]   

            # Reading the image number to be saved
            with open(os.path.join(gt_path, filename), "r") as f:
                line = f.readlines()
                for i in range(len(line)):
                    line[i] = line[i][:-1]
                    # Check if image has already been added:
                    if line[i] not in seen_names:
                        # Append this many names
                        names.append(name)
                        seen_names.append(line[i])
                        image_paths.append(line[i])

    for idx, img in enumerate(image_paths):
        name = names[idx]

        if end_fname[1] != "ok.txt":
            img = img.split(" ")[0][5:]
            image = cv2.imread(images_path + "/" + img + ".jpg")
        else:
            image = cv2.imread(images_path + "/" + img + ".jpg")
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        if gray_image is not None:
            if name in image_dict:
                image_dict[name].append(gray_image)
            else:
                image_dict[name] = [gray_image]
        colour.append(image)
    
    print("Loaded in: {} Images, {} Names, {} Classes".format(len(colour), len(names), len(image_dict)))
    return image_dict, colour, names

In [ ]:
def sift_features(images):
    sift_vectors = {}
    descriptor_list = []
    
    sift = cv2.SIFT_create()
    for key,value in images.items():
        features = []
        for img in value:
            kp, des = sift.detectAndCompute(img,None)
            
            descriptor_list.extend(des)
            features.append(des)
            
        sift_vectors[key] = features
    return [descriptor_list, sift_vectors]

In [ ]:
def kmeans(k, descriptor_list):
    start = time.time()
    
    k_means = KMeans(n_clusters = k, n_init=10)
    k_means.fit(descriptor_list)
    
    end = time.time()
    print("Time Elapsed: {} min".format(round((end - start)/60, 2)))
    
    return k_means

In [ ]:
def create_bow(all_bovw, n_clusters):
    dict_feature = {}
    
    for key,value in all_bovw.items():
        category = []
        
        for img in value:
            histogram = np.zeros(n_clusters)
            
            for each_feature in img:
                ind = k_means.predict([each_feature])
                histogram[ind] += 1
            category.append(histogram)
        dict_feature[key] = category
        
    return dict_feature

In [ ]:
def dict_to_array(diction):
    array = []
    for key, value in diction.items():
        for img in value:
            array.append(img)
    return array

# Code for computing BoW

## Loding in images

Inputs for the function:
- images_path -> the path to folder of the 5k oxford dataset (all images)
- gt_path -> the path to folder of the ground truth files (for queries, and good/ok related images)
- end_fname -> the gt file name endings:
    - ['query.txt', 'none'] -> used to find the query files
    - ['good.txt', 'ok.txt'] -> used to find the good/ok related image files
    
Returns:
- image_dict -> dictionary of the images (whether test or training) split based on class
- colour -> Array of the corresponding images saved in image_dict (these are the colour images)
- names -> Array of the corresponding names for each image

In [ ]:
images_path = r"C:\Users\Sean\Desktop\Image-Retrieval\Oxford code\Oxford dataset\Oxford building images"
gt_path = r"C:\Users\Sean\Desktop\Image-Retrieval\Oxford code\Ground Truth files"

test, test_colour, test_names = load_images_in(images_path, gt_path, ['query.txt', 'none'])
train, train_colour, train_names = load_images_in(images_path, gt_path, ['good.txt', 'ok.txt'])

## Solving SIFT Features

Make sure the train and test data is split into 2 parts!\
I used cv2's SIFT function

Inputs:
- images -> gray_scale images in split dictionary form (filenames as keys)

Returns:
- [descriptor_list, sift_vectors]:
    - descriptor_list -> array of all descriptors stacked without seperation
    - sift_vectors -> dictionary of the descriptors split into classes

In [ ]:
sifts = sift_features(train) 
descriptor_list = sifts[0] 
all_bovw_feature = sifts[1] 

test_bovw_feature = sift_features(test)[1] 

## Clustering

Only clustering the training images sift features!\
Predict the test features as new unseen data, which makes sense to show the algorithm unseen data so it doesn't overfit!\
Using Kmeans clustering as achieves best results (does take long however)

Inputs:
- k -> number of clusters (or the number of visual words wanted)
- descriptor_list -> Array of training sift feature descriptors (test not included!)

Outputs:
- k_means -> the kmeans algorithm (used for prediction of test/query images later)

In [ ]:
n_clusters = 100
k_means = kmeans(n_clusters, descriptor_list) 

## Creating BoW 

Creates a histogram of visual word length (or the number of clusters in the kmeans algorithm)

Inputs:
- all_bovw -> descriptors split by classes
- n_clusters -> number of clusters (number of visual words)

Outputs:
- dict_feature -> Dictionary of histograms split by classes

In [ ]:
bovw_train = create_bow(all_bovw_feature, n_clusters) 
bovw_test = create_bow(test_bovw_feature, n_clusters) 

bovw_train = dict_to_array(bovw_train)
bovw_test = dict_to_array(bovw_test)

## Computing metrics 

Input:  
- train_data -> Training data
- test_data -> Testing data
- train_names -> Corresponding training names
- test_names -> Corresponding testing names
- train_images_as_array -> Corresponding training images (for viewing)
- test_images_as_array -> Corresponding testing images (for viewing)
- k -> How many top k images to show, also to solve precision at that k value
- veiw_option:
    - 0 -> Displays nothing, returns results.
    - 1 -> Displays the top k labelled images, AP and precision@k for each query image

Output:
- avg_precisions -> A list of all AP results
- avg_recalls -> A list of all AR results
- precisionatk -> list of precisions@k for inputted k value

In [ ]:
k = 10
view_option = 1

AP, AR, precisionsatk = image_retrieval_k(bovw_train, bovw_test, train_names, test_names, train_colour, test_colour, k, view_option)

## Computing mAP

Using our Average Precisions from computing metrics, we define the success of the retieval system based on a singular result which is the average of all the AP per query: Named mAP (Mean Average Precision)

In [ ]:
mAP = np.average(AP)
print("mAP: ", mAP)

## Saving to file

Saving the precision at k and average precisions for each query image for a set k value.\
Saves to the working directory as an csv file.

In [ ]:
filename = "gt_oxford5k_BoW_{}".format(n_clusters)
save_data_to_csv(precisionsatk, AP, k, filename)

## mAP values:

The csv file saved doesn't inlcude mAP results (so I have recorded them manualy here)

10        words: 0.24385\
100       words: 0.32494\
1000      words: \
10,000    words: \
100,000   words: \
1,000,000 words: 